<a href="https://colab.research.google.com/github/darkzard05/Cora_dataset_benchmark/blob/main/Spline_conv_by_pytorch_geometric(Cora_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-scatter torch-sparse torch-spline-conv torch_geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 4.9 MB/s 
     |████████████████████████████████| 3.5 MB 46.2 MB/s 
     |████████████████████████████████| 750 kB 29.7 MB/s 
     |████████████████████████████████| 407 kB 5.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=671a0126ec9d76e5043109dbfaa7a40be23d363ec5ab1f61a821ef29df9b64fb
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv

print(torch.__version__)

1.11.0+cu113


In [3]:
dataset_name = 'Cora'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name, 
                    transform=T.TargetIndegree()
                    )

Processing...
Done!


In [4]:
data = dataset[0]
print(f'dataset name: {dataset_name}')
print(f'number of nodes: {data.num_nodes} // number of edges: {data.num_edges} // number of classes: {dataset.num_classes}')
print(f'number of node features: {data.num_node_features} // number of edge features: {data.num_edge_features}')
print(data.x, data.edge_index, data.edge_attr, data.y)
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset name: Cora
number of nodes: 2708 // number of edges: 10556 // number of classes: 7
number of node features: 1433 // number of edge features: 1
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]]) tensor([[0.0179],
        [0.0238],
        [0.0179],
        ...,
        [0.1964],
        [0.0238],
        [0.0238]]) tensor([3, 4, 4,  ..., 3, 3, 3])
tensor(140) tensor(500) tensor(1000)


In [5]:
class Model(torch.nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.layer1 = SplineConv(data.num_node_features, hidden, dim=1, kernel_size=2)
        self.layer2 = SplineConv(hidden, dataset.num_classes, dim=1, kernel_size=2)

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = self.layer1(x, edge_index, edge_attr)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.layer2(x, edge_index, edge_attr)
        x = F.elu(x)
        return F.log_softmax(x, dim=1)

In [6]:
print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden=16).to(device)
print(model)
model.reset_parameters()
print({i:list(j.shape) for i, j in model.named_parameters()})

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    loss = criterion(pred[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(mask):
    model.eval()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    correct = pred.argmax(dim=1)[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc

best_acc = 0
epochs = 1000
for epoch in range(1, epochs+1):
    loss = train()
    train_acc = test(data.train_mask)
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    if best_acc < test_acc:
        best_acc = test_acc
        print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')

cuda is available
Model(
  (layer1): SplineConv(1433, 16, dim=1)
  (layer2): SplineConv(16, 7, dim=1)
)
{'layer1.weight': [2, 1433, 16], 'layer1.bias': [16], 'layer1.lin.weight': [16, 1433], 'layer2.weight': [2, 16, 7], 'layer2.bias': [7], 'layer2.lin.weight': [7, 16]}
epoch: 1 | loss: 1.9533 | train_acc: 0.8929 | val_acc: 0.538 | test_acc: 0.542
epoch: 2 | loss: 1.7775 | train_acc: 0.9571 | val_acc: 0.578 | test_acc: 0.555
epoch: 3 | loss: 1.5752 | train_acc: 0.9643 | val_acc: 0.608 | test_acc: 0.594
epoch: 4 | loss: 1.4034 | train_acc: 0.9929 | val_acc: 0.63 | test_acc: 0.633
epoch: 5 | loss: 1.1961 | train_acc: 1.0000 | val_acc: 0.652 | test_acc: 0.646
epoch: 6 | loss: 1.0230 | train_acc: 1.0000 | val_acc: 0.656 | test_acc: 0.673
epoch: 7 | loss: 0.8819 | train_acc: 1.0000 | val_acc: 0.688 | test_acc: 0.704
epoch: 8 | loss: 0.7233 | train_acc: 1.0000 | val_acc: 0.706 | test_acc: 0.721
epoch: 9 | loss: 0.6402 | train_acc: 1.0000 | val_acc: 0.734 | test_acc: 0.74
epoch: 10 | loss: 0.5